In [1]:
distance = 15
doubleplane = 30
energy = 600
neutron = 1
physicss = ["inclxx", "bic", "bert"]

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import joblib
import matplotlib.pyplot as plt
import ROOT
import particle

pd.options.display.min_rows = 100
pd.options.display.max_rows = 100
ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


In [3]:
excluded_particles = [22, 111]

In [4]:
cache = dict()


def pdg_to_name(pdg):
    if pdg in cache:
        return cache[pdg]
    try:
        result = particle.Particle.from_pdgid(pdg).name
    except:
        try:
            npdg = pdg // 10 * 10
            result = particle.Particle.from_pdgid(npdg).name
        except:
            result = str(pdg)
    cache[pdg] = result
    return result

In [5]:
def process(physics):
    simufile = f"output/{distance}m_{doubleplane}dp_{energy}AMeV_{neutron}n_{physics}.simu.root"

    tfile = ROOT.TFile.Open(simufile)
    ttree = tfile.evt

    nEvents = ttree.GetEntries()

    data = []
    for evt in ttree:
        z = -1.0
        secondaries = []
        for nTrack, track in enumerate(evt.MCTrack):
            # Only those created by primary neutrons, no gammas/pi0, no late particles, in the detector
            if (
                (track.GetMotherId() == 0)
                and (track.GetPdgCode() not in excluded_particles)
                and (track.GetStartT() < 1e-5)
                and (track.GetStartZ() > distance * 100)
            ):
                secondaries.append(track.GetPdgCode())
        secondaries.sort()
        rs = "+".join([pdg_to_name(pdg) for pdg in secondaries])
        data.append(rs)

    tfile.Close()
    del tfile

    counts = Counter(data)
    df = pd.DataFrame([(k, v) for k, v in counts.items()], columns=["Reaction Products", "Counts"])
    df.sort_values(by=["Counts"], ascending=[False], inplace=True)
    df["Frequency [%]"] = df["Counts"] / nEvents * 100
    df = df.reset_index(drop=True)
    return df

In [6]:
for physics in physicss:
    reactions = process(physics)
    print(f"QGSP_{physics.upper()}_HP {reactions.shape[0]}")
    reactions.to_csv(f"results/Reactions-QGSP_{physics.upper()}_HP.csv")
    display(reactions)

QGSP_INCLXX_HP 3283


,Reaction Products,Counts,Frequency [%]
0,n+n+p+D2+He4+He4,7586,7.586
1,n+n+n+p+p+He4+He4,6658,6.658
2,n+p+B11,4870,4.870
3,,4739,4.739
4,n+n+C11,4201,4.201
5,n+n+n+p+p+D2+D2+He4,2663,2.663
6,n+He4+He4+He4,2331,2.331
7,n+p+He4+Li7,2285,2.285
8,n+n+p+He4+Li6,1947,1.947
9,n+n+p+B10,1830,1.830


QGSP_BIC_HP 2614


,Reaction Products,Counts,Frequency [%]
0,n+n+n+p+p+He4+He4,12502,12.502
1,n+n+p+D2+He4+He4,5655,5.655
2,n+p+B11,5433,5.433
3,,4686,4.686
4,n+n+C11,4150,4.150
5,pi-+n+n+p+p+p+He4+He4,2704,2.704
6,n+n+n+n+p+p+p+D2+He4,2389,2.389
7,n+n+p+B10,2130,2.130
8,n+n+p+He4+Li6,2114,2.114
9,n+He4+He4+He4,2110,2.110


QGSP_BERT_HP 2275


,Reaction Products,Counts,Frequency [%]
0,n+p+B11,9545,9.545
1,n+n+C11,7797,7.797
2,n+n+n+p+p+He4+He4,6082,6.082
3,,4752,4.752
4,n+n+p+B10,4120,4.120
5,n+n+n+p+p+Be8,3202,3.202
6,pi-+n+p+C11,2570,2.570
7,n+n+p+D2+He4+He4,2349,2.349
8,n+C12,1756,1.756
9,n+He4+Be8,1750,1.750
